In [1]:
import usb.core
import usb.util 
import time 
from adafruit_servokit import ServoKit


## PID Ball on Plate Controller

In this work we present a PID controller for a ball in a balancing plate. The PID is implemented in a discrete domain, where the setpoint of the ball is at position 0. 

The PID adjusts the controller of the output $Q(t)$ which is based on the following equation

$e(t) = T_{sp} - T_{pv}$

$Q(t) = Q_{bias} + K_ce(t) + \frac{K_c}{\tau_I}\int_{0}^{t} e(t)\,dt - K_c\tau_D \frac{d(T_{PV})}{dt} $

1. $Q_{bias}$ constants typically set to the value of Q(t) when the controller is first turned on. For the first experiment it will be $Q_{bias} = 0$. For $Q(t)$ to change its value it should be $Q'(t) = Q(t) - Q_{bias}$ 
2. **Intregral** 
The continous integral term is approximated a discrete form as the summation of th error multiplied by the sample time. 
$\frac{K_c}{\tau_I}\int_{0}^{t} e(t)\,dt \approx\frac{K_c}{\tau_I} \sum\limits_{i=1}^n e_i(t)Δt$

3. **Derivative** The derivative term is a finite approximation. In this case uses the prior values of PV to get the current slope of the system. 
$\frac{dT_{PV}}{dt}$$ \approx \frac{T_{PV n}-T_{PV n-1}}{\Delta t}$

The discrete form of the PID is the following:

$Q(t) = Q_{bias} + K_ce(t) + \frac{K_c}{\tau_I} \sum\limits_{i=1}^n e_i(t)Δt - K_c\tau_D \frac{d(T_{PV})}{dt} $

---
PID Function Python:

**INPUT** 
* sp = Setpoint
* cv = current value
* pv = previous value
* iErr = integral error
* dt = measurement time

**OUTPUT**
* out = output of the PID
* P = Proportional Control 
* I = Integral Control 
* D = Derivative Control  


In [2]:
def pid(sp, cv, pv, iErr, dt):
  #K Proportional, Integral and Derivative
  KD = 0.0 #Kc
  KI = 0.0 #Kc/tauI
  KP = 0.1 #Kc*tauD
  #Initial values 
  ser0 = 0
  serHigh = 180 
  serLow = 0 
  #Errors
  err = sp - pv
  iErr = iErr + KI*err*dt #Integral
  dErr = (cv - pv)/dt #derivative
  #PID components
  #P = KP * err
  #I = iErr
  #D = KD*dpv

  u = KP*err + KI*iErr + KD*dErr

  if u > 180:
    u = 180
  if u < 0: 
    u = 0
  


  #ser = ser0 + P + I + D
  #if ser > serHigh or ser < serLow:
  #  I = I - KI*err*dt
  #  ser = max(serLow,min(serHigh, ser))
  return [u]

#Touchscreen Initialization
def touchScreen_init(): 
  dev = usb.core.find(idVendor = 0x04d8, idProduct = 0x0c02)
  ep_in = dev[0].interfaces()[0].endpoints()[0]
  ep_out = dev[0].interfaces()[0].endpoints()[1]
  intf = dev[0].interfaces()[0].bInterfaceNumber
  dev.reset()
  #Check if the touchscreen has been found 
  if dev is None: 
    raise ValueError('Device not found')
  else:
    print(dev)
  #Claim the USB interface from the computer 
  if dev.is_kernel_driver_active(intf):
    dev.detach_kernel_driver(intf)
    usb.util.claim_interface(dev, intf)
  return dev, ep_in, ep_out

  
def touchScreen_data(dev, ep_in, ep_out):
  #Screen constants 
  screen_x_lim_up = 1870
  screen_x_lim_down = 1917
  screen_y_lim_up = 3920
  screen_y_lim_down = 1666
  try: 
    data = dev.read(ep_in.bEndpointAddress, ep_in.wMaxPacketSize) #Collected data from the touchscreen
    X_coordinate = (((data[2] - 8)*256 + data[1])*-1)/(screen_x_lim_down)
    Y_coordinate = (((data[4] - 8)*256 + data[3])*-1)/(screen_y_lim_down)
  except usb.core.USBError as e: 
    data = None 

  return [X_coordinate, Y_coordinate]

In [3]:
touchScreen_init()

USBError: [Errno 13] Access denied (insufficient permissions)

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import time 

Kc = 0.9
tauI = 0.1
tauD = 0.1
iErr = 0 
dt = 0.1


n = 1000 #Number of set point 
tm = np.linspace(0, n-1, n)
#Meausrement from the touch screen
M = np.zeros(n) #Measurement from the servo
Q = np.zeros(n)   #Setpoint to 0 
SP = np.zeros(n)  #Setpoint to 1
SP[300:] = -1.0

Xd = 0 
Yd = 0

Qbias = 0.0 

while True: 
  #Read the touchscreen 
  #Mx an My




for i in range(n): 
  #Get measurement from the touch screen 
  M[i] =  
  #Change the value of Q 

  [Q[i], P, I, D] = pid(SP[i], M[i], M[max(0, i-1)], iErr, dt)

  [Q[i], P] = p(SP[i])

  #Send the value of Q to the servo 
  print('Servo', 'Ball Position', 'Setpoint')
  print(f'{Q[i]:7.2f},{M[i]:2.7f},{SP[i]:7.2f}')
  time.sleep(1)

data = np.vstack()

# Create Figure
# Plot 1
plt.figure(figsize=(10,7))
ax = plt.subplot(2,1,1)
ax.grid()
plt.plot(X,SP,'k-',label=r'$SP$ SP')
plt.plot(X,M,'r.',label=r'$M$ PV')
plt.ylabel(r' ($^oC$)')
plt.legend(loc=2)

# Plot 2
ax = plt.subplot(2,1,2)
ax.grid()
plt.plot(X,Q,'b-',label=r'$Q_1$')
plt.ylabel(r'Ball position (%)')
plt.xlabel('Time (s)')
plt.legend(loc=1)
plt.show()


# ServoKit Controller


In [22]:
kit = ServoKit(channels=16)
kit.servo[0].angle=110
kit.servo[2].angle=90

## Reference

Markdown | Preview
--- | ---
`**bold text**` | **bold text**
`*italicized text*` or `_italicized text_` | *italicized text*
`` `Monospace` `` | `Monospace`
`~~strikethrough~~` | ~~strikethrough~~
`[A link](https://www.google.com)` | [A link](https://www.google.com)
`![An image](https://www.google.com/images/rss.png)` | ![An image](https://www.google.com/images/rss.png)


---
Headings are rendered as titles.

```markdown
# Section 1
# Section 2
## Sub-section under Section 2
### Sub-section under the sub-section under Section 2
# Section 3
```

# Section 1
# Section 2
## Sub-section under Section 2
### Sub-section under the sub-section under Section 2
# Section 3

The table of contents, available on the left side of Colab, is populated using at most one section title from each text cell.

---

```markdown
>One level of indentation
```

>One level of indentation


```markdown
>>Two levels of indentation
```

>>Two levels of indentation

---

Code blocks

````
```python
print("a")
```
````

```python
print("a")
```

---

Ordered lists:
```markdown
1. One
1. Two
1. Three
```
1. One
1. Two
1. Three

---

Unordered lists:
```markdown
* One
* Two
* Three
```
* One
* Two
* Three

---

Equations:

```markdown
$y=x^2$

$e^{i\pi} + 1 = 0$

$e^x=\sum_{i=0}^\infty \frac{1}{i!}x^i$

$\frac{n!}{k!(n-k)!} = {n \choose k}$

$A_{m,n} =
 \begin{pmatrix}
  a_{1,1} & a_{1,2} & \cdots & a_{1,n} \\
  a_{2,1} & a_{2,2} & \cdots & a_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  a_{m,1} & a_{m,2} & \cdots & a_{m,n}
 \end{pmatrix}$
```

$y=x^2$

$e^{i\pi} + 1 = 0$

$e^x=\sum_{i=0}^\infty \frac{1}{i!}x^i$

$\frac{n!}{k!(n-k)!} = {n \choose k}$

$A_{m,n} =
 \begin{pmatrix}
  a_{1,1} & a_{1,2} & \cdots & a_{1,n} \\
  a_{2,1} & a_{2,2} & \cdots & a_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  a_{m,1} & a_{m,2} & \cdots & a_{m,n}
 \end{pmatrix}$


 ---

Tables:
```markdown
First column name  | Second column name 
-------------------|------------------
Row 1, Col 1       | Row 1, Col 2 
Row 2, Col 1       | Row 2, Col 2 

```

First column name  | Second column name 
-------------------|------------------
Row 1, Col 1       | Row 1, Col 2 
Row 2, Col 1       | Row 2, Col 2 

---

Horizontal rules:
```markdown
---
```

---




## Differences between Colab Markdown and other Markdown dialects

Colab uses [marked.js](https://github.com/chjj/marked) and so is similar but not quite identical to the Markdown used by Jupyter and Github.

Colab supports (MathJax) $\LaTeX$ equations like Jupyter, but does not allow HTML tags in the Markdown. Colab does not support some GitHub additions like emojis and to-do checkboxes.

If HTML must be included in a Colab notebook, see the [%%html magic](/notebooks/basic_features_overview.ipynb#scrollTo=qM4myQGfQboQ).

## Useful references

* [Github Markdown basics](https://help.github.com/articles/markdown-basics/)
* [Github flavored Markdown](https://help.github.com/articles/github-flavored-markdown/)
* [Original Markdown spec: Syntax](http://daringfireball.net/projects/markdown/syntax)
* [Original Markdown spec: Basics](http://daringfireball.net/projects/markdown/basics)
* [marked.js library used by Colab](https://github.com/chjj/marked)
* [LaTex mathematics for equations](https://en.wikibooks.org/wiki/LaTeX/Mathematics)